### DataSet Definition

In [8]:
from sklearn.datasets import make_classification

In [9]:
x,y = make_classification(n_samples = 1000, n_features = 10, n_informative=8, n_redundant=2, random_state=0)

In [10]:
x[0]

array([ 0.59748132,  2.03702154,  0.63388218,  2.89010949,  1.30628617,
       -0.52502013,  0.19653851, -0.58915931, -0.5775684 , -2.07947151])

In [11]:
y[0]

1

## Model Definition

In [12]:
import tensorflow as tf
from tensorflow import keras

In [18]:
def create_model():
  model = keras.Sequential()
  model.add(keras.layers.Dense(units =16, activation= 'relu'))
  model.add(keras.layers.Dense(units= 1, activation = 'sigmoid'))

  model.compile(optimizer = tf.optimizers.Adam(), loss = tf.losses.binary_crossentropy, metrics = ['accuracy'])

  return model

### Cross Validation

In [14]:
n_split = 5

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

kf = StratifiedKFold(n_splits=n_split, shuffle = True)

In [22]:
acc_total = []

for index_train, index_validation in kf.split(x, y):
  x_train = x[index_train]
  y_train = y[index_train]

  x_vald = x[index_validation]
  y_vald = y[index_validation]


  model = create_model()
  model.fit(x_train, y_train, epochs = 20, batch_size =64, validation_data= (x_vald, y_vald), verbose = 2 )

  y_pred = model.predict(x_vald)
  y_pred = (y_pred > 0.5).astype('int32')

  acc = accuracy_score(y_vald, y_pred)
  acc_total.append(acc)


Epoch 1/20
13/13 - 3s - loss: 1.0970 - accuracy: 0.4837 - val_loss: 0.9784 - val_accuracy: 0.5050 - 3s/epoch - 224ms/step
Epoch 2/20
13/13 - 0s - loss: 0.9586 - accuracy: 0.5188 - val_loss: 0.8561 - val_accuracy: 0.5450 - 124ms/epoch - 10ms/step
Epoch 3/20
13/13 - 0s - loss: 0.8343 - accuracy: 0.5612 - val_loss: 0.7538 - val_accuracy: 0.6050 - 275ms/epoch - 21ms/step
Epoch 4/20
13/13 - 0s - loss: 0.7313 - accuracy: 0.6062 - val_loss: 0.6693 - val_accuracy: 0.6250 - 154ms/epoch - 12ms/step
Epoch 5/20
13/13 - 0s - loss: 0.6483 - accuracy: 0.6625 - val_loss: 0.5983 - val_accuracy: 0.6950 - 281ms/epoch - 22ms/step
Epoch 6/20
13/13 - 0s - loss: 0.5801 - accuracy: 0.7275 - val_loss: 0.5424 - val_accuracy: 0.7250 - 163ms/epoch - 13ms/step
Epoch 7/20
13/13 - 0s - loss: 0.5263 - accuracy: 0.7550 - val_loss: 0.4995 - val_accuracy: 0.7800 - 110ms/epoch - 8ms/step
Epoch 8/20
13/13 - 0s - loss: 0.4830 - accuracy: 0.7887 - val_loss: 0.4666 - val_accuracy: 0.8150 - 88ms/epoch - 7ms/step
Epoch 9/20
13

In [24]:
acc_total

[0.89, 0.93, 0.905, 0.93, 0.885]

### Grid Search

In [26]:
x,y  = make_classification(n_samples = 1000, n_features = 20, n_informative = 10, n_redundant=5, random_state= 0 )

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
x_train, x_test,y_train, y_test = train_test_split(x,y, test_size = 0.2 )

In [29]:
x_train.shape, y_train.shape

((800, 20), (800,))

In [31]:
x_test.shape, y_test.shape

((200, 20), (200,))

### Create Model

In [32]:
import tensorflow as tf
from tensorflow import keras

In [58]:
def create_model(hidden_size = 16 , optimizer = 'adam'):
  model = keras.Sequential()
  model.add(keras.layers.Dense(units = hidden_size, activation = 'relu',input_shape=(None, 20)))
  model.add(keras.layers.Dense(units = 1, activation = 'sigmoid'))

  model.compile(optimizer = optimizer, loss= 'binary_crossentropy', metrics = ['accuracy'])

  return model


In [59]:
from sklearn.model_selection import GridSearchCV

In [60]:
!pip install scikeras

In [61]:
from scikeras.wrappers import KerasClassifier

In [62]:
model  = KerasClassifier(model = create_model, loss = 'binary_crossentropy', epochs=10, batch_size = 32, verbose = 0)

params = {
    'model__hidden_size':[16, 32 ,64],
    'model__optimizer': ['adam', 'sgd']
}

In [63]:
grid = GridSearchCV(estimator=model, param_grid=params, cv = 3)
grid_result = grid.fit(x_train, y_train)

In [64]:
best_params= grid_result.best_params_

In [65]:
best_params

{'model__hidden_size': 64, 'model__optimizer': 'adam'}

In [66]:
grid_result.best_score_

0.8112228135882779

### Random Search

In [67]:
from sklearn.model_selection import RandomizedSearchCV

In [68]:
rs = RandomizedSearchCV(estimator = model, param_distributions=params, n_iter = 4)

In [69]:
rs_result = rs.fit(x_train, y_train)

In [70]:
rs_result.best_score_

0.8125

In [71]:
rs_result.best_params_

{'model__optimizer': 'adam', 'model__hidden_size': 64}

End!